In [33]:
#libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [39]:
df = pd.read_csv('train.csv')

---
## Transfomations:

In [40]:
# rename the columns names using lower case and replace space with '_'

def rename_columns(df):
    df.columns = [columns.lower().replace(" ","_") for columns in df.columns]
    return df

df = rename_columns(df)

In [41]:
# add 1st_flr_sf and 2nd_flr_sf and create a new column total_sf
def create_total_sf(df):
    df['total_sf'] = df['1st_flr_sf'] + df['2nd_flr_sf']
    return df

df = create_total_sf(df)

In [42]:
def all_qual(df):
    df['bsmt_qual'].fillna('Fa',inplace=True)
    df['bsmt_qual'] = df['bsmt_qual'].map({'Po':2,'Fa':2,'TA':3,'Gd':4,'Ex':5})
    df['all_qual'] = df['bsmt_qual'] + df['overall_qual']
    return df
df = all_qual(df)

In [43]:
def bsmt_qual_alter(df):
    df['bsmt_qual_test1'] = df['bsmt_qual'].map(lambda x: 'Fa' if str(x) == 'nan' or x == 'Po' else x)
    df.drop('bsmt_qual',axis = 1,inplace=True)
    return df
df = bsmt_qual_alter(df)

In [44]:
def bsmt_full_bath_na(df):
    df['bsmt_full_bath'].fillna(0,inplace=True)
    return df
    
df = bsmt_full_bath_na(df)

In [45]:
def garage_cars_na(df):
    df = df[~df['garage_cars'].isnull()]
    return df
df = garage_cars_na(df)

In [46]:
def room_avg_size(df):
    df['room_avg_size'] = df['total_sf'] / (df['totrms_abvgrd'] + df['full_bath'] + 0.5*df['half_bath'] + df['kitchen_abvgr'])
    return df
df = room_avg_size(df)

In [47]:
def total_baths(df):
    df['total_baths'] = df['full_bath'] + 0.5*df['half_bath'] + 0.5* df['bsmt_full_bath'].fillna(0) + 0.5 *0.5* df['bsmt_half_bath'].fillna(0)
    return df
df = total_baths(df)

In [253]:
# def central_air_convert(df):
#     df['central_air'] = df['central_air'].map({'Y':1,'N':0})
#     return df
# df = central_air_convert(df)

---
## First try


In [48]:
df_1st = df[['overall_qual','year_built', 'exter_qual', 'bsmt_qual_test1', 'central_air','total_sf','bsmt_full_bath','full_bath','kitchen_qual','totrms_abvgrd','fireplaces','garage_cars','saleprice','room_avg_size']]

In [49]:
df_1st_dummy = pd.get_dummies(df_1st,drop_first=True)

In [50]:
X = df_1st_dummy.drop('saleprice', axis=1)
y = df_1st_dummy['saleprice']

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)

In [51]:
linreg_1 = LinearRegression()
linreg_1.fit(X_train,y_train)
linreg_1.score(X_train,y_train)

0.8265385234213605

In [114]:
linreg_1.score(X_test,y_test)

0.8530189256712726

In [52]:
cross_val_score(linreg_1,X_train,y_train,cv=10).mean()

0.8125124826975478

---
### compare

In [54]:
df_compare = df[['overall_qual','year_built', 'exter_qual', 'bsmt_qual_test1', 'central_air','total_sf','bsmt_full_bath','full_bath','kitchen_qual','totrms_abvgrd','fireplaces','garage_cars','saleprice']]

In [55]:
df_comp_dummy = pd.get_dummies(df_compare,drop_first=True)

In [56]:
X_comp = df_comp_dummy.drop('saleprice', axis=1)
y_comp = df_comp_dummy['saleprice']

X_train_comp,X_test_comp,y_train_comp,y_test_comp = train_test_split(X_comp,y_comp,random_state=42)

In [57]:
linreg_comp = LinearRegression()
linreg_comp.fit(X_train_comp,y_train_comp)

linreg_comp.score(X_train_comp,y_train_comp)

0.8233548875985597

In [113]:
linreg_comp.score(X_test_comp,y_test_comp)

0.8558706037472356

In [115]:
cross_val_score(linreg_comp,X_train_comp,y_train_comp,cv=10)

array([0.81687355, 0.85817739, 0.87370906, 0.842806  , 0.80654737,
       0.88871304, 0.86907093, 0.74961456, 0.57576861, 0.84398492])

---
### 3rd try

In [87]:
df_3rd = df[['all_qual', 'exter_qual', 'central_air','total_sf','kitchen_qual','totrms_abvgrd','fireplaces','garage_cars','saleprice','room_avg_size']]

In [88]:
df_3rd_dummies = pd.get_dummies(df_3rd,drop_first=True)

In [89]:
X_3rd = df_3rd_dummies.drop('saleprice',axis=1)
y_3rd = df_3rd_dummies['saleprice']

In [90]:
X_train_3rd,X_test_3rd,y_train_3rd,y_test_3rd = train_test_split(X_3rd,y_3rd,random_state=42)

In [91]:
linreg_3rd = LinearRegression()
linreg_3rd.fit(X_train_3rd,y_train_3rd)

linreg_3rd.score(X_train_3rd,y_train_3rd)

0.8131831831403985

In [112]:
linreg_3rd.score(X_test_3rd,y_test_3rd)

0.8462187744508909

In [94]:
cross_val_score(linreg_3rd,X_train_3rd,y_train_3rd,cv=10).mean()

0.8023681097053945

In [93]:
df_3rd.corr()

,all_qual,total_sf,totrms_abvgrd,fireplaces,garage_cars,saleprice,room_avg_size
all_qual,1.000000,0.556290,0.353668,0.372151,0.610605,0.817400,0.492171
total_sf,0.556290,1.000000,0.805640,0.461949,0.497681,0.705500,0.747194
totrms_abvgrd,0.353668,0.805640,1.000000,0.311710,0.368034,0.503990,0.265716
fireplaces,0.372151,0.461949,0.311710,1.000000,0.310687,0.471034,0.452987
garage_cars,0.610605,0.497681,0.368034,0.310687,1.000000,0.648220,0.380024
saleprice,0.817400,0.705500,0.503990,0.471034,0.648220,1.000000,0.592318
room_avg_size,0.492171,0.747194,0.265716,0.452987,0.380024,0.592318,1.000000


---
### 4th

In [227]:
df_4 = df[['all_qual', 'central_air','total_sf','kitchen_qual','totrms_abvgrd','fireplaces','garage_cars','room_avg_size','saleprice']]

In [228]:
df_4_dummies = pd.get_dummies(df_4,drop_first=True)

In [229]:
#df_4_dummies= sc.fit_transform(df_4_dummies)

In [230]:
X_4 = df_4_dummies.drop('saleprice',axis=1)
# X_4['total_sf'] = np.log(X_4['total_sf'])
y_4 = df_4_dummies['saleprice']

In [231]:
X_train_4,X_test_4,y_train_4,y_test_4 = train_test_split(X_4,y_4,random_state=42)

In [232]:
linreg_4 = LinearRegression()
linreg_4.fit(X_train_4,y_train_4)

linreg_4.score(X_train_4,y_train_4)

0.8001526643120968

In [233]:
cross_val_score(linreg_3rd,X_train_3rd,y_train_3rd,cv=10)

array([0.79740497, 0.8457347 , 0.86309721, 0.81172187, 0.79805092,
       0.87058115, 0.86240229, 0.7514779 , 0.60065859, 0.8225515 ])

In [234]:
linreg_4.score(X_test_4,y_test_4)

0.8477975842062146

In [235]:
df_4.corr()

,all_qual,total_sf,totrms_abvgrd,fireplaces,garage_cars,room_avg_size,saleprice
all_qual,1.000000,0.556290,0.353668,0.372151,0.610605,0.492171,0.817400
total_sf,0.556290,1.000000,0.805640,0.461949,0.497681,0.747194,0.705500
totrms_abvgrd,0.353668,0.805640,1.000000,0.311710,0.368034,0.265716,0.503990
fireplaces,0.372151,0.461949,0.311710,1.000000,0.310687,0.452987,0.471034
garage_cars,0.610605,0.497681,0.368034,0.310687,1.000000,0.380024,0.648220
room_avg_size,0.492171,0.747194,0.265716,0.452987,0.380024,1.000000,0.592318
saleprice,0.817400,0.705500,0.503990,0.471034,0.648220,0.592318,1.000000


In [165]:
df_4.describe()

,all_qual,total_sf,totrms_abvgrd,fireplaces,garage_cars,saleprice,room_avg_size
count,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000
mean,9.656585,1493.654146,6.436098,0.591220,1.776585,181484.609268,159.731917
std,2.009037,497.685300,1.560576,0.638539,0.764537,79275.121591,29.058405
min,3.000000,334.000000,2.000000,0.000000,0.000000,12789.000000,74.285714
25%,8.000000,1124.250000,5.000000,0.000000,1.000000,129812.500000,139.777778
50%,9.000000,1440.000000,6.000000,1.000000,2.000000,162500.000000,156.857143
75%,11.000000,1728.000000,7.000000,1.000000,2.000000,214000.000000,175.156682
max,15.000000,5642.000000,15.000000,4.000000,5.000000,611657.000000,364.000000


In [150]:
from sklearn.preprocessing import StandardScaler

In [151]:
sc=StandardScaler()

---
### Regulization

In [177]:
ela_net_2 = ElasticNet(alpha=0.1, l1_ratio=0.5)
ela_net_2.fit(X_train,y_train)
ela_net_2.score(X_train,y_train)

0.8055545704949139

In [179]:
cross_val_score(ela_net_2,X_train,y_train,cv=10).mean()

0.7947892494458273

In [243]:
df['central_air']

0       Y
1       Y
2       Y
3       Y
4       Y
       ..
2046    Y
2047    Y
2048    Y
2049    Y
2050    Y
Name: central_air, Length: 2050, dtype: object

In [240]:
np.mean([1,5,10,100])

29.0

In [ ]:
from sklearn

0.09090909090909091

In [309]:
df_5 = df[['all_qual','year_built','total_sf','total_baths','totrms_abvgrd','fireplaces','garage_cars','room_avg_size','saleprice','kitchen_qual']]

In [310]:
df_5 = pd.get_dummies(df_5,drop_first=True)

In [311]:
X = df_5.drop('saleprice',axis=1)
y = df_5['saleprice']

In [361]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42)

In [313]:
from sklearn.preprocessing import StandardScaler

In [314]:
sc = StandardScaler()

In [315]:
X_train_sc = sc.fit_transform(X_train)
y_train_sc = sc.fit_transform(np.array(y_train).reshape(-1,1))

In [316]:
linreg_5 = LinearRegression()
linreg_5.fit(X_train_sc,y_train_sc)

LinearRegression()

In [317]:
linreg_5.score(X_train_sc,y_train_sc)

0.8122270729908615

In [319]:
cross_val_score(linreg_5,X_train_sc,y_train_sc,cv=10)

array([0.84109936, 0.77991898, 0.86926016, 0.78737203, 0.78091635,
       0.85403489, 0.83432964, 0.79323274, 0.64286968, 0.82735684])

In [190]:
from sklearn.linear_model import ElasticNet

In [321]:
ela_net = ElasticNet(alpha=0.1, l1_ratio=0.3)
ela_net.fit(X_train_sc,y_train_sc)
ela_net.score(X_train_sc,y_train_sc)

0.7854772675436748

In [205]:
ela_net.score(X_test_4,y_test_4)

0.8320043607471559

In [206]:
cross_val_score(ela_net,X_train_4,y_train_4,cv=10)

array([0.78016174, 0.8233082 , 0.83030895, 0.80310553, 0.76799542,
       0.84021107, 0.82055511, 0.74027721, 0.62660314, 0.78539743])

In [345]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [331]:
pipe = Pipeline([('sc', StandardScaler()),
                  ('Ela_Net', ElasticNet())])

In [332]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('sc', StandardScaler()), ('Ela_Net', ElasticNet())])

In [333]:
pipe.score(X_train,y_train)

0.7718946670868578

In [334]:
pipe.get_params()

{'memory': None,
 'steps': [('sc', StandardScaler()), ('Ela_Net', ElasticNet())],
 'verbose': False,
 'sc': StandardScaler(),
 'Ela_Net': ElasticNet(),
 'sc__copy': True,
 'sc__with_mean': True,
 'sc__with_std': True,
 'Ela_Net__alpha': 1.0,
 'Ela_Net__copy_X': True,
 'Ela_Net__fit_intercept': True,
 'Ela_Net__l1_ratio': 0.5,
 'Ela_Net__max_iter': 1000,
 'Ela_Net__normalize': False,
 'Ela_Net__positive': False,
 'Ela_Net__precompute': False,
 'Ela_Net__random_state': None,
 'Ela_Net__selection': 'cyclic',
 'Ela_Net__tol': 0.0001,
 'Ela_Net__warm_start': False}

In [369]:
pipe_params = {
    'sc__with_mean':[True,False],
    'sc__with_std':[True,False],
    'Ela_Net__alpha':[0.06,0.07,0.08,0.09],
    'Ela_Net__l1_ratio':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
}

In [370]:
pipe_gridsearch = GridSearchCV(pipe, # What is the model we want to fit?
                                 pipe_params, # What is the dictionary of hyperparameters?
                                 cv=5, # What number of folds in CV will we use?
                                 verbose=1)

In [371]:
pipe_gridsearch.fit(X_train, y_train);

Fitting 5 folds for each of 160 candidates, totalling 800 fits


In [372]:
pipe_gridsearch.score(X_train,y_train)

0.811165214671828

In [373]:
pipe_gridsearch.score(X_test,y_test)

0.8381987630318408

In [374]:
pipe_gridsearch.best_estimator_

Pipeline(steps=[('sc', StandardScaler()),
                ('Ela_Net', ElasticNet(alpha=0.08, l1_ratio=0.8))])